In [93]:
import pandas as pd
import ast
import json
# AWS Python SDK
import boto3
# When running on SageMaker, need execution role
from sagemaker import get_execution_role
role = get_execution_role()

In [94]:
s3 = boto3.resource('s3')

In [95]:
# Declare bucket name, remote file, and destination
my_bucket = 'nodel-ag'
prefix_list = ['ex_1','ex_2','ex_3','ex_4']

In [96]:
def build_dataframe(bucket, prefix):
    result_list = []
    for obj in bucket.objects.filter(Prefix=f"{prefix}/"):
        data = obj.get()['Body'].read().decode()
        if data:
            if data.find("\n") != -1:
                row = data.split("\n")
                result_list.append(json.loads(row[0]))
                result_list.append(json.loads(row[0]))
                # result_list.append(ast.literal_eval(row[0]))
                # result_list.append(ast.literal_eval(row[1]))
            else:
                # result_list.append(ast.literal_eval(data))
                result_list.append(json.loads(data))
    return pd.DataFrame(result_list)  

In [97]:
bucket = s3.Bucket(my_bucket)
for prefix in prefix_list:
    df = build_dataframe(bucket, prefix)
    df.to_csv(prefix+'.csv', sep=',', index=False)